Explore TwitterAPI
====

Based on this code: http://geduldig.github.io/TwitterAPI/faulttolerance.html

In [21]:
reset -fs

In [32]:
from time import sleep

from credentials import credentials
from ftfy import fix_text
from textblob import TextBlob
from TwitterAPI import TwitterAPI

In [23]:
creds = credentials.require(['access_token', 
                             'access_token_secret', 
                             'consumer_key',
                             'consumer_secret'])

api = TwitterAPI(creds.consumer_key,
                 creds.consumer_secret,
                 creds.access_token,
                 creds.access_token_secret)

In [24]:
def tweets(query):
    "A generator of tweets"
    while True:
        try:
            iterator = api.request('search/tweets', {'q':query}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield(item['text'])
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [25]:
t = tweets(query='pizza')

In [38]:
for _ in range(10):
    blob = TextBlob(fix_text(next(t)))
    print(blob.raw)
    print(blob.tags)
    
    print('#'*40)
    sleep(1)

pizza pizza pizza is bae
[('pizza', 'NN'), ('pizza', 'NN'), ('pizza', 'NN'), ('is', 'VBZ'), ('bae', 'JJ')]
########################################
Pi day has been a day of friendship and shitty pie for the past couple of years this year is hopefully pizza and a couple pals
[('Pi', 'NNP'), ('day', 'NN'), ('has', 'VBZ'), ('been', 'VBN'), ('a', 'DT'), ('day', 'NN'), ('of', 'IN'), ('friendship', 'NN'), ('and', 'CC'), ('shitty', 'JJ'), ('pie', 'NN'), ('for', 'IN'), ('the', 'DT'), ('past', 'JJ'), ('couple', 'NN'), ('of', 'IN'), ('years', 'NNS'), ('this', 'DT'), ('year', 'NN'), ('is', 'VBZ'), ('hopefully', 'RB'), ('pizza', 'JJ'), ('and', 'CC'), ('a', 'DT'), ('couple', 'NN'), ('pals', 'NNS')]
########################################
RT @adam_bronicki: The priorities in life are     Eating pizza z
[('RT', 'NNP'), ('@', 'NNP'), ('adam_bronicki', 'NN'), ('The', 'DT'), ('priorities', 'NNS'), ('in', 'IN'), ('life', 'NN'), ('are', 'VBP'), ('Eating', 'VBG'), ('pizza', 'NN'), ('z', 'NN')]
###########

---
__The internet also has Rainbow Butterfly Unicorn Kittens:__

![](https://s-media-cache-ak0.pinimg.com/736x/bf/3f/4c/bf3f4c4e4cbc909f957f939bb6bc7cc6.jpg)